In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from pathlib import Path

# ==========================================
# 1. CARGA DE DATOS (SANTIAGO CENTRO)
# ==========================================
pr_dir = Path('pr')
files = sorted(pr_dir.glob('CR2MET_pr_v2.5_day_*.nc'))
if not files:
    raise RuntimeError('No se encontraron archivos CR2MET en pr/')

ds = xr.open_mfdataset([str(p) for p in files], combine='by_coords')
lat_sc, lon_sc = -33.45, -70.66
pr_sc = ds["pr"].sel(lat=lat_sc, lon=lon_sc, method="nearest").sel(time=slice("1979-01-01", "2021-12-31"))

# Definir días secos (< 1mm es el estándar)
dry_bool = (pr_sc < 1.0).values
time_values = pr_sc.time.values

# ==========================================
# 2. FUNCIÓN PARA IDENTIFICAR DRY SPELLS
# ==========================================
def get_dry_spells(time_index, dry_array):
    dry = np.asarray(dry_array, dtype=bool)
    x = np.r_[False, dry, False]
    dx = np.diff(x.astype(int))
    starts = np.where(dx == 1)[0]
    ends = np.where(dx == -1)[0]
    lengths = ends - starts
    
    return pd.DataFrame({
        "start": pd.to_datetime(time_index[starts]),
        "duration": lengths
    })

df_spells = get_dry_spells(time_values, dry_bool)
df_spells["year"] = df_spells["start"].dt.year

# ==========================================
# 3. ESTADÍSTICAS POR VENTANAS DE 5 AÑOS
# ==========================================
# Tal como pidió el profe: "agregar primero la data y después calcular"
years = np.arange(1979, 2018) # Ventanas: 79-83, 80-84... hasta 17-21
rolling_data = []

for start_y in years:
    end_y = start_y + 4
    # Agregamos todos los eventos del bloque de 5 años
    window_spells = df_spells[(df_spells["year"] >= start_y) & (df_spells["year"] <= end_y)]["duration"]
    
    if not window_spells.empty:
        rolling_data.append({
            "year_center": start_y + 2,
            "mean": window_spells.mean(),
            "max": window_spells.max(),
            "p99": np.percentile(window_spells, 99),
            "tL": np.percentile(window_spells, 95) # Aproximación de tL por percentil alto
        })

df_stats = pd.DataFrame(rolling_data).set_index("year_center")

# ==========================================
# 4. CÁLCULO DE TENDENCIAS (DÍAS/DÉCADA)
# ==========================================
metrics = ["mean", "max", "p99", "tL"]
tendencias = {}

for m in metrics:
    y = df_stats[m].values
    x = df_stats.index.values
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
    tendencias[m] = slope * 10 # Convertir de días/año a días/década

# ==========================================
# 5. GRÁFICO FINAL
# ==========================================
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True)
axes = axes.flatten()
titles = ["Promedio Duración", "Duración Máxima", "Percentil 99", "Escala de Corte (tL)"]

for i, m in enumerate(metrics):
    ax = axes[i]
    ax.plot(df_stats.index, df_stats[m], 'o-', label='Datos (5yr window)', color='darkblue')
    
    # Línea de tendencia
    x_range = df_stats.index
    y_trend = stats.linregress(x_range, df_stats[m].values).intercept + \
              stats.linregress(x_range, df_stats[m].values).slope * x_range
    ax.plot(x_range, y_trend, 'r--', label=f'Tendencia: {tendencias[m]:.2f} días/década')
    
    ax.set_title(titles[i], fontweight='bold')
    ax.set_ylabel("Días")
    ax.legend()
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("Resumen de Tendencias (Días/Década):")
for k, v in tendencias.items():
    print(f"{k}: {v:.2f} días/década")